# Collapsing traceroutes
this code
 * looks up all the unique traceroutes (according to hash).
 * prepends source site IP
 * For each IP (not first or last) make a list of all previous and following IPs (and how often it happened)
 * find all IPs that had matching lists.
 * makes list of all the IPs, make a list of IP equivalences. 


In [1]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
my_index = "network_weather-2017.9.*"

my_query = {}

##  Select your link

In [2]:
# CHANGE SO IT RETREIVES ONLY UNIQUE hashes.

# sS='UC'
# srcSiteOWDServer = "192.170.227.160"
# srcSiteThroughputServer = "192.170.227.162"

sS='CERN-PROD'
srcSiteOWDServer = "128.142.223.247"
srcSiteThroughputServer = "128.142.223.246"

# dS='IU'
# destSiteOWDServer = "149.165.225.223"
# destSiteThroughputServer = "149.165.225.224"

# dS='UIUC'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.96.9"

# dS='ICCN'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.126.132"

dS='pic'
destSiteOWDServer = "193.109.172.188"
destSiteThroughputServer = "193.109.172.187"

## Get the data from Elasticsearch, and store the record based on its type

In [10]:
my_query = {
    "size":1,
    "_source": {
        "include": [ 'src','dest','hops','hash' ]
    },
    'query':{
        'bool':{
            'must':[
                {'range': {'timestamp': {'gte': '20170915T000000Z', 'lt': '20170930T000000Z'}}},
                {'term': {'_type': 'traceroute'}},
#                         {'bool':
#                             {'should':[
#                                 {'term': {'src': srcSiteOWDServer}},
#                                 {'term': {'src': srcSiteThroughputServer}},
#                                 {'term': {'src': destSiteOWDServer}},
#                                 {'term': {'src': destSiteThroughputServer}}
#                             ]}
#                         }
#                         ,
#                         {'bool':
#                             {'should':[
#                                 {'term': {'dest': destSiteOWDServer}},
#                                 {'term': {'dest': destSiteThroughputServer}},
#                                 {'term': {'dest': srcSiteOWDServer}},
#                                 {'term': {'dest': srcSiteThroughputServer}}
#                             ]}
#                         }
            ]

        }
    },
    "aggs": {
        "grouped_by_hash": {
          "terms": {  "field": "hash", "size":10000 }, #
          "aggs": {
              "top_hash_hits": {
                  "top_hits": {
                      "sort": [ { "_score": { "order": "desc" } } ],
                      "size": 1
                  }
              }
          }
       }
    }
}

results = es.search(body=my_query, index=my_index, request_timeout=12000)
print(len(results['aggregations']['grouped_by_hash']['buckets']))

buckets=results['aggregations']['grouped_by_hash']['buckets']
results=[]
# c=0
for b in buckets:
#     if c==0: 
#         print(b)
#         c+=1
    if b['doc_count']>20: # we don't want very short jumps
        results.append(b['top_hash_hits']['hits']['hits'][0])

10000


In [9]:
AllIPs=set()
uniquePaths={}
count = 0
for res in results:
    if count<1: print(res)
    count += 1
    # for data before Oct. 15 2016
    s=res['_source']['src']
    d=res['_source']['dest']
    hops=[s]
    hops += res['_source']['hops']
    h=res['_source']['hash']
    if h not in uniquePaths: 
        if hops[-1]==None: hops[-1]=d
        uniquePaths[h] = hops
        for h in hops:
            if h!=None:
                AllIPs.add(h)
print('uniquePaths:',len(uniquePaths))
print('AllIPs:', len(AllIPs))

{'_score': 2.0, '_source': {'destVO': 'ATLAS', 'src': '72.36.96.15', 'srcProduction': False, 'MA': '72.36.96.15', 'destSite': 'Nebraska', 'n_hops': 8, 'max_rtt': 30.422, 'srcSite': 'MWT2', 'destProduction': True, 'hops': ['72.36.96.1', '130.126.1.109', '72.36.80.30', '146.57.252.194', '146.57.252.222', '146.57.252.225', '146.57.253.217', '129.93.5.165'], 'dest': '129.93.5.165', 'src_host': 'mwt2-ps03.campuscluster.illinois.edu', 'srcVO': 'ATLAS', 'hash': 2705678586548672572, 'timestamp': 1505433635000, 'ipv6': False, 'rtts': [1.003, 0.201, 11.575999999999999, 11.67, 15.879000000000001, 30.422, 25.311, 24.854999999999997], 'dest_host': 'perfsonar.unl.edu', 'ttls': [1, 2, 3, 4, 5, 6, 7, 8]}, '_index': 'network_weather-2017.9.15', '_type': 'traceroute', '_id': 'AV6C1nbJbY2Pr9ADWBSu'}
uniquePaths: 10
AllIPs: 49


### Look for one off differences

In [ ]:
AllSegs={}
for uP in uniquePaths.values():
    l=len(uP)
    #print(uP, l)
    for si in range(1,l-1):
        if uP[si]==None: continue
        if uP[si] not in AllSegs: AllSegs[uP[si]]=set()
        AllSegs[uP[si]].add(uP[si-1])
        AllSegs[uP[si]].add(uP[si+1])
print("non-edge devices:", len(AllSegs))

In [ ]:
simpleIPs=set()
toLook=0
for s in AllSegs:
    if len(AllSegs[s])<3:
        simpleIPs.add(s)
        simpleIPs=simpleIPs.union(AllSegs[s])
    else: toLook+=1
    if len(AllSegs[s])>2: 
        print(s, AllSegs[s])
print ('simple:',len(simpleIPs) , 'toLook:', toLook)

In [ ]:
#print(EquivalentDevices)
groups=[]
for s in EquivalentDevices:
    for d in EquivalentDevices[s]:
        #print (s,d,EquivalentDevices[s][d])
        groups.append(set(EquivalentDevices[s][d]))

In [ ]:
print(groups)

In [ ]:
l=len(groups)
for i in range(l):
    for ip in groups[i]:
        for j in range(i+1,l):
            if ip in groups[j]:
                groups[i]=groups[i].union(groups[j])
                groups[j]=set()
                continue

In [ ]:
for g in groups:
    if len(g):
        print(g)